## CODIGO PRINCIPAL

En este notebook se incluyen las ejecuciones de los programas desarrollados en el proyecto de forma secuencial.

En concreto este código está dedicado a los capítulos del trabajo indicados a continuación:

    3. Sistema, diseño y desarrollo
    4. Experimentos realizados y resultados

#### Importar funciones y módulos requeridos

El módulo *nimgenetics.py* contiene las funciones necesarias para ejecutar este código.

In [ ]:
import os
import sys

sys.path.append('./my_module/')
import nimgenetics

### 3. SISTEMA, DISEÑO Y DESARROLLO

     
Directorios importantes:
     
- folder_path: carpeta donde se ubican los informes de NIMGenetics.
- folder_result: carpeta donde se irán guardando los resultados de las ejecuciones de los programas.
- pathI: path de la carpeta "Identificados" de los informes que cumplen el patrón de selección.
- image_path: carpeta donde se almacenan las imágenes generadas a partir de los informes.
- textos: carpeta donde se almacenan los textos generados a partir de las imágenes.

In [ ]:
folder_path = 'path_to_folder_root'
folder_result = 'path_to_folder_result'
pathI = 'path_to_folder_identificados'
image_path = 'path_to_images'
textos = 'path_to_textos'

#### 3.2 Obtención de informes clínicos de Gestlab
Informes filtrados y renombrados.

#### 3.2.1 Mapeo de los informes localizados en folder_path
La función load_files es utilizada para mapear los informes (PDF) dentro de folder_path.

In [ ]:
# l_path contiene el path absoluto de todos los informes
l_path = nimgenetics.load_files(root_dir=folder_path)[0]
# l_pdf es el mapeo individual de cada informe
l_pdf = nimgenetics.load_files(root_dir=folder_path)[1]

#### 3.2.2 Obtención de los identificadores reales de los informes
La función getIDs realiza una consulta sql a Gestlab para obtener los identificadores reales.

In [ ]:
# id_peticion es una lista con los ID reales obtenidos tras la decodificación hexadecimal
id_peticion = []
for pdf in l_pdf:
    # split por guion bajo
    s_pdf = pdf.split('_')
    # transformación del segundo componente en hexadecimal del identificador
    hex_id = int(s_pdf[1],16)
    id_peticion.append(hex_id)

# ejecución de la función getIDs
r_id = nimgenetics.getIDs(id_list=id_peticion)

#### 3.2.3 Almacenamiento de los informes con su ID correcto
Selección de informes que cumplen un patrón específico: identifiedPDF.

Informes cuyo identificador es desconocido: emptyPDF.

In [ ]:
# Generar una lista que almacene todos los identificadores provenientes de r_id
# l_names a diferencia de la anterior no es una lista de listas
l_names = []   
for row in r_id:
    try:
        # Primer bloque: extraer identificadores que cumplan la siguiente estructura
        l_names.append(row[0][0])
    except:
        # Segundo bloque: identificadores que devuelven elementos vacios
        # Estos informes se almacenarán con su identificador hexadecimal original
        s_row = str(row)
        if s_row == '[]':
            # Almacenamos en l_names los identificadores reemplazados con el literal 'sin_id'
            sr_row = s_row.replace('[]', 'sin_id')
            l_names.append(sr_row)


# Función utilizada para almacenar los informes que cumplen un patrón de selección pre-establecido
# También se almacenan los informes que están repetidos y los descartados que no cumplen el patrón
# con la finalidad de no perder información
nimgenetics.identifiedPDF(folder_path=folder_result, list_id=l_names, list_path=l_path)

# Función para almacenar los informes cuyo identificador no ha sido recuperado de la base de datos
# Estos informes son guardados con el identificador hexadecimal original
nimgenetics.emptyPDF(folder_path=folder_result, list_id=l_names, list_path=l_path, list_pdf=l_pdf)

#### 3.3 Conversión de los informes a texto plano para la extracción de terminología médica

In [ ]:
# pdfPath contiene el path absoluto de todos los pdf 
# este paso es necesario para ejecutar el siguiente programa: pdfToImage
pdfPath = nimgenetics.absolutePath(dpath=pathI)

#### 3.3.1 Generación de imágenes en formato PNG a partir de los informes escaneados
Este procedimiento involucra la ejecución de la función pdfToImage.

In [ ]:
# Ejecución de la función pdfToImage
nimgenetics.pdfToImage(f_path=folder_result, pdf_list=pdfPath)

#### 3.3.2 Generación de texto plano a partir de imágenes PNG
Este procedimiento involucra la ejecución de dos funciones: makeCorpus y ocrToImage.

In [ ]:
# Ejecución de la función makeCorpus la cual integra en su código ocrToImage
# Necesario image_path como parámetro para la correcta ejecución de la función
# El resultado de este proceso guarda los resultados en la carpeta textos
nimgenetics.makeCorpus(f_path=folder_result, image_path=image_path)

### 4. EXPERIMENTOS REALIZADOS Y RESULTADOS

Directorios importantes:

- cutext_results: carpeta que incluye los resultados de la ejecución de CUTEXT.
- cutext_corpus: carpeta que alberga el corpus elaborado al procesar los resultados de CUTEXT.
- megadict: path donde se localiza el megadiccionario.
- lookup_corpus: path donde almacenar el corpus generado por la estrategia lookup.
- corpus_global: path donde guardar el corpus global elaborado a partir de cutext y lookup.
- lookup_corpus_uncleaned: path donde se almacena el corpus de lookup sin proceso de curación.

In [ ]:
cutext_results = 'path_to_save_cutext_results'
cutext_corpus = 'path_to_save_cutext_corpus'
megadict = 'path_to_megadict'
lookup_corpus = 'path_to_lookup_corpus'
corpus_global = 'path_to_merge_corpus'
lookup_corpus_uncleaned = 'path_to_lookup_corpusUncleaned'

In [ ]:
# path_textos contiene el path absoluto de los textos
path_textos = nimgenetics.absolutePath(dpath=textos)

#### 4.2 Extracción de terminología mediante CUTEXT
Ejecutar la función getCutextResults que integra la funcionalidad de CUTEXT.

In [ ]:
# path a la carpeta in de cutext
cutext_in = 'path_to_cutext_in_folder'
# path del output generado por cutext
out = 'path_to_cutext_output_folder'
# El output de cutext se almacenará en cutext_results
# ejecución de la función que integra la funcionalidad de cutext
nimgenetics.getCutextResults(cutext_in=cutext_in, textos_ori=path_textos, old=out, new=cutext_results)

#### 4.2.1 Elaboración de un corpus final a partir de los resultados CUTEXT
La función cutextCorpus procesa los resultados generados en el apartado anterior.

In [ ]:
# path absoluto de los resultados de cutext
l_cutext = nimgenetics.absolutePath(dpath=cutext_results)

# Ejecución del programa que genera el corpus de cutext
# El resultado se almacena en cutext_corpus
nimgenetics.cutextCorpus(lista_cutext=l_cutext, path_to_words=cutext_corpus)

#### 4.3 Extracción de terminología mediante estrategia lookup

La función lookup genera un corpus elaborado utilizando esta estrategia.

In [ ]:
# Almacenamos el corpus de lookup en lookup_corpus
nimgenetics.lookup(textos_ori=path_textos, megadict=megadict, path_to_words=lookup_corpus)

#### 4.5 Elaboración de un corpus global a partir de cutext y lookup
La función mergeCorpus implica la construcción de un corpus que alberga la terminología extraída mediante cutext y la estrategia lookup.

In [ ]:
# Este proceso genera el corpus global y se almacena en corpus_global
nimgenetics.mergeCorpus(p_cutext=cutext_corpus, p_lookup=lookup_corpus, path_to_words=corpus_global)

#### 4.5.1 Representación de clínica sobre una matriz de distribución terminológica
Diseñar una función capaz de mapear los términos extraídos al paciente correspondiente

In [ ]:
# Construir la matriz a partir de un dataFrame almacenado como un documento excel
nimgenetics.mapeoTerminos(l_textos=path_textos, infile_global=corpus_global)

#### 4.5.2 Selección de 500 términos para el mapeo manual a HPO
Función que genera dos diccionarios relativo al conteo global de términos y los 500 términos para el mapeo manual a HPO.

In [ ]:
all_terms = nimgenetics.subsetManual(c_uncleaned=lookup_corpus_uncleaned, c_global=corpus_global)

El programa devuelve el conteo global -*all_terms*- gracias a la función items del módulo *FreqDist* de NLTK. 

Esta lista es utilizada para generar la imagen de la distribución terminológia en función de la frecuencia de los términos.

In [ ]:
# -all_terms- diccionario que integra la lista completa de términos
# Los términos aparecen en formato key, value
# key = término
# value = número de veces que aparece el término

# Primera categoría:
values_500 = []
values_500_50 = []
values_50 = []

for key, value in all_terms:
    
    if value > 500:
        values_500.append(key)
    elif value <= 500 and value >= 50:
        values_500_50.append(key)
    else:
        values_50.append(key)

names = ['500', '500-50', '50']
values = [len(values_500), len(values_500_50), len(values_50)]

# Segunda categoría:
# Diccionario con las palabras <= 50 repeticiones en el corpus global
dict2 = {k: v for k, v in all_terms if v <= 50}
d2 = dict2.items()

v_50_25 = []
v_25_10 = []
v_10 = []

for key, value in d2:
    
    if value <= 50 and value > 25:
        v_50_25.append(key)
    elif value <= 25 and value > 10:
        v25_10.append(key)
    else:
        v_10.append(key)

names1 = ['50-25', '25-10', '10']
values1 = [len(v_50_25), len(v_25_10), len(v_10)]

# Tercera categoría:
# Diccionario con las palabras <= 10 repeticiones en el corpus global
dict3 = {k: v for k, v in all_terms if v <= 10}
d3 = dict3.items()

v_10_5 = []
v_5_3 = []
v_3 = []

for key, value in d3:
    
    if value <= 10 and value > 5:
        v_10_5.append(key)
    elif value <= 5 and value > 3:
        v_5_3.append(key)
    else:
        v_3.append(key)
        
names2 = ['10-5', '5-3', '3']
values2 = [len(v_10_5), len(v_5_3), len(v_3)]

# Cuarta categoría:
# Diccionario con las palabras <= 3 repeticiones en el corpus global
dict4 = {k: v for k, v in all_terms if v <= 3}
d4 = dict4.items()

v_3_ = []
v_2_ = []
v_1_ = []

for key, value in d4:
    
    if value == 3:
        v_3_.append(key)
    elif value == 2:
        v_2_.append(key)
    else:
        v_1_.append(key)

names3 = ['3', '2', '1']
values3 = [len(v_3_), len(v_2_), len(v_1_)]

# Plotear la distribución de categorías terminológicas
fig = plt.figure(figsize=(12,10))
ax0 = fig.add_subplot(111)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax0.spines['top'].set_color('none')
ax0.spines['bottom'].set_color('none')
ax0.spines['left'].set_color('none')
ax0.spines['right'].set_color('none')
ax0.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

ax0.set_ylabel('Conteo de términos', fontsize=12, labelpad=20)
ax0.set_xlabel('Frecuencia de términos', fontsize=12, labelpad=20)

ax1.bar(names, values)
ax1.set_ylim(0, 4500)

ax2.bar(names1, values1)
ax2.set_ylim(0, 4500)

ax3.bar(names2, values2)
ax3.set_ylim(0, 4500)

ax4.bar(names3, values3)
ax4.set_ylim(0, 4500)

fig.suptitle('Categorias Terminológicas', fontsize=20)
fig.savefig('distribucion de terminología.png')